EL PLANTEAMIENTO CON RESPECTO A LA PRIMERA VERSIÓN ES DIFERENTE: A LA VISTA DE LOS ERRORES QUE SE PRODUCEN CON LA DESCARGA DEL TOTAL RETURN POR PARTES PRINCIPALMENTE DE LOS ÍNDICES DE RENTA FIJA, LO QUE SE HACE EN ESTE ALGORITMO ES DESCARGAR TODA LA SERIE TEMPORAL DE PRECIOS, ALINEARLAS TEMPORALMENTE Y POSTERIORMENTE CALCULAR LAS CARTERAS Y EL PERFORMANCE. DE HECHO, ESTA VERSIÓN SE HA PLANTEADO DEBIDO A QUE ALGUNOS ÍNDICES NO ESTÁN REPRESENTADOS EN PORT Y LOS RESULTADOS QUE SE OBTIENEN NO SON REALES.

In [13]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
import bqviz as bqv
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']
fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [14]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 3 #months
lambda_risk = 0 # lambda_risk = 1 Full HRP | lambda_risk = 0 Full HAP (Recursive bisection based on Expected Return)

In [15]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [16]:
#HRAP generation funtions

import scipy.cluster.hierarchy as sch

#------------------------------------------------------------------------------
#The distance Matrix is define as :
def correlDist(corr): 
    # A distance matrix based on correlation, where 0<=d[i,j]<=1
    # This is a proper distance metric
    dist=((1-corr)/2.)**.5 # distance matrix
    return dist

#------------------------------------------------------------------------------
def getQuasiDiag(link):
    # Sort clustered items by distance
    link=link.astype(int)
    sortIx=pd.Series([link[-1,0],link[-1,1]])
    numItems=link[-1,3] # number of original items
    while sortIx.max()>=numItems:
        sortIx.index=range(0,sortIx.shape[0]*2,2) # make space
        df0=sortIx[sortIx>=numItems] # find clusters
        i=df0.index;j=df0.values-numItems
        sortIx[i]=link[j,0] # item 1
        df0=pd.Series(link[j,1],index=i+1)
        sortIx=sortIx.append(df0) # item 2
        sortIx=sortIx.sort_index() # re-sort
        sortIx.index=range(sortIx.shape[0]) # re-index
    return sortIx.tolist()

#------------------------------------------------------------------------------
#Función Recursive Bisection:
def getRecBipart_harp(cov,sortIx,xReturns=None, riskaversion=0.5, minWt = 0.001, maxWt = 1.0):
    # Compute HRP alloc
    w=pd.Series(1,index=sortIx)
    cItems=[sortIx] # initialize all items in one cluster
    while len(cItems)>0:
        cItems=[i[int(j):int(k)] for i in cItems for j,k in ((0,len(i)/2), \
                                                   (len(i)/2,len(i))) if len(i)>1] # bi-section
        for i in range(0,len(cItems),2): # parse in pairs
            cItems0=cItems[i] # cluster 1
            cItems1=cItems[i+1] # cluster 2
            cVar0=getClusterVar(cov,cItems0)
            cVar1=getClusterVar(cov,cItems1)
            if xReturns is None:
                alpha=1-cVar0/(cVar0+cVar1)
                w[cItems0]*=alpha # weight 1 
                w[cItems1]*=1-alpha # weight 2
            else:
                eVar0 = xReturns[cItems0].mean()   # Expected Returns of group 1
                eVar1 = xReturns[cItems1].mean()   # Expected Returns of group 2
                alpha = 1. - cVar0 / (cVar0+cVar1) # HRPi Relative Variance - HRP 'alpha'
                xprtn = 0. + eVar0 / (eVar0+eVar1) # HAPi Expected Return Relative
                
                # High Risk Aversion results in more stock weight from variance vs. returns.
                w[cItems0] *= riskaversion*alpha + (xprtn*(1.-riskaversion))           # HARPi Wt1
                w[cItems1] *= riskaversion*(1.-alpha) + (1.-xprtn)*((1.-riskaversion)) # Wt2                          
        
    return w 

#------------------------------------------------------------------------------
#Función para calcular la varianza por cada cluster:
def getClusterVar(cov,cItems):
    # Compute variance per cluster
    cov_=cov.loc[cItems,cItems] # matrix slice
    w_=getIVP(cov_).reshape(-1,1)
    cVar=np.dot(np.dot(w_.T,cov_),w_)[0,0]
    return cVar

#------------------------------------------------------------------------------
#Función para calcular la inversa de la varianza del portfolio:   
def getIVP(cov,**kargs):
    # Compute the inverse-variance portfolio
    ivp=1./np.diag(cov)
    ivp/=ivp.sum()
    return ivp

#------------------------------------------------------------------------------
def HierarchicalRiskAlphaParity(dataFrame, riskaversion): #DF de la serie de retorno acumulado y el coeficiente de aversión al riesgo
    expReturns = dataFrame.mean()
    expReturns = getxprtnnZeroOne(expReturns)
    corr = dataFrame.corr()
    dist = correlDist(corr)
    link=sch.linkage(dist,method='single', metric='euclidean')
    #Almacenamos los indices ordenados:
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist() # recover labels
    cov = dataFrame.cov() #Matriz de covarianzas
    hrap=getRecBipart_harp(cov,sortIx,xReturns=expReturns, riskaversion=riskaversion) # Capital allocation   
    hrap = pd.DataFrame(data = {'Assets': hrap.index.tolist(),'Weights': hrap.values.tolist()})    
    
    return hrap

#------------------------------------------------------------------------------
def getxprtnnZeroOne(expReturns=pd.Series):
    # Rescale expected returns from 0 to 1
    if expReturns is None:
        pass
    else:
        idx=expReturns.index
        expReturns = zscore(expReturns)
        expReturns = expReturns/max(abs(expReturns))/2.0+0.5
        expReturns = expReturns / max(abs(expReturns))
        # in case when max(abs()) was -ve, then new max will be < 1.0
        expReturns = pd.Series(expReturns, index=idx)
    return expReturns

def zscore(a, axis=0, ddof=0, keepNaN=False):
    """ NAN Stable Z-Scores"""
    try:
        idx = a.index
    except:
        idx = None
    a = np.asanyarray(a)
    mns = np.nanmean(a, axis=axis)
    sstd = np.nanstd(a=a, axis=axis, ddof=ddof)
    if axis and mns.ndim < a.ndim:
        res = ((a - np.expand_dims(mns, axis=axis)) /
               np.expand_dims(sstd, axis=axis))
    else:
        res = (a - mns) / sstd # RESult
    if not keepNaN:
        res = np.nan_to_num(res) # Default set to zero where was NaN
    res = pd.Series(res, index=idx)
    return res

In [17]:
#Get the whole price series por all the indexes of the ETFs
download_start_date = start_date - relativedelta(months=12)
download_end_date = end_date + relativedelta(months=rotation)

price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(u.list(etf_under_idx_list), {'Price': price})
res = bq.execute(request)
price_series = res[0].df().reset_index()

#Pivot the price series to get the df in a way that the rows are dates and columns indexes prices
align_price_series = price_series.pivot(index='DATE', columns='ID', values='Price').reset_index().dropna().reset_index().drop(['index'], axis=1)

#Add to the price series df an auxiliar date 
year_list = pd.DatetimeIndex(align_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(align_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
align_price_series['aux_date'] = date_list  

#Get the price series for HFRIFOF Index (the returned data is in a monthly basis)
price = f.dropna(d.px_last(start = download_start_date, end = download_end_date))
request = bql.Request(index_to_replicate, {'Price': price})
res = bq.execute(request)
index_price_series = res[0].df().reset_index()

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
index_price_series = index_price_series.drop(['ID', 'CURRENCY'], axis=1)
index_price_series = index_price_series.rename(index=str, columns={'Price': index_to_replicate})
year_list = pd.DatetimeIndex(index_price_series['DATE']).year.tolist()
month_list = pd.DatetimeIndex(index_price_series['DATE']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
index_price_series['aux_date'] = date_list

grouped_price_series = pd.DataFrame(data = {})

list_aux_dates = align_price_series['aux_date'].unique().tolist()
for i_date in list_aux_dates:
    aux_mat = align_price_series.loc[align_price_series['aux_date'].isin([i_date]), align_price_series.columns != 'DATE']
    i_idx = aux_mat.index.tolist()
    aux_mat = aux_mat.loc[i_idx[-1]].to_frame().T
    
    if i_date == list_aux_dates[0]:
        grouped_price_series = aux_mat
        
    else:
        grouped_price_series = grouped_price_series.append(aux_mat, ignore_index=True)

grouped_price_series = pd.merge(index_price_series, grouped_price_series, how = 'left', on=['aux_date', 'aux_date'])

index_universe = [index_to_replicate]+etf_under_idx_list

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)

    test_data = align_price_series.loc[(align_price_series['DATE'] >= aux_date_st) & (align_price_series['DATE'] <= aux_date)]
    test_data = test_data.loc[:,((align_price_series.columns != 'aux_date') & (align_price_series.columns != 'DATE')).tolist()]

    #Calculate the return
    test_data = (test_data.pct_change().fillna(0) + 1).cumprod() - 1    
    
#     #Calculate the cumulative total return
#     test_data = (test_data.pct_change().fillna(0) + 1).cumprod() - 1    
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    port = HierarchicalRiskAlphaParity(test_data, lambda_risk)
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date,'security': port['Assets'].tolist(), 'weight': port['Weights'].tolist()})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date,'security': port['Assets'].tolist(), 'weight': port['Weights'].tolist()})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)
    
port_df

2006-01-01 00:00:00
2006-04-01 00:00:00
2006-07-01 00:00:00
2006-10-01 00:00:00
2007-01-01 00:00:00
2007-04-01 00:00:00
2007-07-01 00:00:00
2007-10-01 00:00:00
2008-01-01 00:00:00
2008-04-01 00:00:00
2008-07-01 00:00:00
2008-10-01 00:00:00
2009-01-01 00:00:00
2009-04-01 00:00:00
2009-07-01 00:00:00
2009-10-01 00:00:00
2010-01-01 00:00:00
2010-04-01 00:00:00
2010-07-01 00:00:00
2010-10-01 00:00:00
2011-01-01 00:00:00
2011-04-01 00:00:00
2011-07-01 00:00:00
2011-10-01 00:00:00
2012-01-01 00:00:00
2012-04-01 00:00:00
2012-07-01 00:00:00
2012-10-01 00:00:00
2013-01-01 00:00:00
2013-04-01 00:00:00
2013-07-01 00:00:00
2013-10-01 00:00:00
2014-01-01 00:00:00
2014-04-01 00:00:00
2014-07-01 00:00:00
2014-10-01 00:00:00
2015-01-01 00:00:00
2015-04-01 00:00:00
2015-07-01 00:00:00
2015-10-01 00:00:00
2016-01-01 00:00:00
2016-04-01 00:00:00
2016-07-01 00:00:00
2016-10-01 00:00:00
2017-01-01 00:00:00
2017-04-01 00:00:00
2017-07-01 00:00:00
2017-10-01 00:00:00
2018-01-01 00:00:00
2018-04-01 00:00:00


date        security    weight
0  2006-01-01       RTY Index  0.083023
1  2006-01-01  BFU5TRUU Index  0.107480
2  2006-01-01    FTAW01 Index  0.124132
3  2006-01-01  NDUEACWF Index  0.068627
4  2006-01-01    M1WOEV Index  0.095910
5  2006-01-01  MIMUEMRN Index  0.232773
6  2006-01-01   NDDUE15 Index  0.117361
7  2006-01-01  NDDUEAFE Index  0.109274
8  2006-01-01    BCIW1A Index  0.026141
9  2006-01-01  IDCOT3TR Index  0.035278
0  2006-04-01       RTY Index  0.156200
1  2006-04-01  BFU5TRUU Index  0.068608
2  2006-04-01   NDDUE15 Index  0.129610
3  2006-04-01  MIMUEMRN Index  0.140676
4  2006-04-01    FTAW01 Index  0.071752
5  2006-04-01  NDUEACWF Index  0.126592
6  2006-04-01    M1WOEV Index  0.153618
7  2006-04-01  NDDUEAFE Index  0.108942
8  2006-04-01    BCIW1A Index  0.018674
9  2006-04-01  IDCOT3TR Index  0.025326
0  2006-07-01  BFU5TRUU Index  0.054939
1  2006-07-01       RTY Index  0.114267
2  2006-07-01  MIMUEMRN Index  0.241438
3  2006-07-01    FTAW01 Index  0.062751
4  2006-07-01  NDUEACWF Index  0.066176
5  2006-07-01   NDDUE15 Index  0.142387
6  2006-07-01    M1WOEV Index  0.163246
7  2006-07-01  NDDUEAFE Index  0.128809
8  2006-07-01    BCIW1A Index  0.015760
9  2006-07-01  IDCOT3TR Index  0.010227
..        ...             ...       ...
0  2018-07-01  IDCOT3TR Index  0.000000
1  2018-07-01  BFU5TRUU Index  0.031143
2  2018-07-01       RTY Index  0.156909
3  2018-07-01    BCIW1A Index  0.046020
4  2018-07-01  MIMUEMRN Index  0.145002
5  2018-07-01   NDDUE15 Index  0.122480
6  2018-07-01  NDDUEAFE Index  0.147580
7  2018-07-01    M1WOEV Index  0.191116
8  2018-07-01    FTAW01 Index  0.075647
9  2018-07-01  NDUEACWF Index  0.084102
0  2018-10-01  IDCOT3TR Index  0.000000
1  2018-10-01  BFU5TRUU Index  0.073635
2  2018-10-01       RTY Index  0.191865
3  2018-10-01    BCIW1A Index  0.042821
4  2018-10-01  MIMUEMRN Index  0.090309
5  2018-10-01    FTAW01 Index  0.165879
6  2018-10-01  NDUEACWF Index  0.188373
7  2018-10-01    M1WOEV Index  0.163901
8  2018-10-01   NDDUE15 Index  0.030218
9  2018-10-01  NDDUEAFE Index  0.053000
0  2019-01-01       RTY Index  0.222322
1  2019-01-01    M1WOEV Index  0.094734
2  2019-01-01   NDDUE15 Index  0.081257
3  2019-01-01  NDDUEAFE Index  0.044977
4  2019-01-01    FTAW01 Index  0.061993
5  2019-01-01  NDUEACWF Index  0.149604
6  2019-01-01    BCIW1A Index  0.123850
7  2019-01-01  MIMUEMRN Index  0.000000
8  2019-01-01  BFU5TRUU Index  0.128176
9  2019-01-01  IDCOT3TR Index  0.093088

[530 rows x 3 columns]

In [18]:
#Securities don't follow the same order in different time periods
port_df = port_df.sort_values(by=['date', 'security'])

#Add to the portfolio df an auxiliar date 
year_list = pd.DatetimeIndex(port_df['date']).year.tolist()
month_list = pd.DatetimeIndex(port_df['date']).month.tolist()
date_list = []
for j in range(len(year_list)):
    date_list += [str(year_list[j]) + '-' + str(month_list[j])]
port_df['aux_date'] = date_list  

#Get the price and consecuently the number of shares for each moment of the rebalancing date
list_reb_dates = port_df['aux_date'].unique().tolist()

price_0_list = []
for i_date in list_reb_dates:
    aux_port_df = port_df.loc[port_df['aux_date'].isin([i_date])]
    price_0_list += grouped_price_series.loc[grouped_price_series['aux_date'].isin([i_date])][aux_port_df['security'].tolist()].values[0].tolist()
port_df['Price'] = price_0_list
port_df['N_Assets_type'] = port_df['weight']/port_df['Price']

#For the first date the calculated number of shares it's OK, but for the rest of the dates it must be take into account the value of the portfolio in that moment
aux_df_real_n_assets = port_df.loc[port_df['aux_date'].isin([list_reb_dates[0]])][['aux_date', 'N_Assets_type']]
for i_prev_date,i_date in zip(list_reb_dates[:-1], list_reb_dates[1:]):
    n_assets = np.array(aux_df_real_n_assets.loc[aux_df_real_n_assets['aux_date'].isin([i_prev_date])]['N_Assets_type'].tolist())
    cur_price = np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['Price'].tolist())
    val = sum(n_assets*cur_price)
    cur_n_assets = val*np.array(port_df.loc[port_df['aux_date'].isin([i_date])]['weight'].tolist())/cur_price
    
    aux_df_real_n_assets = aux_df_real_n_assets.append(pd.DataFrame(data={'aux_date': [i_date]*len(cur_n_assets), 'N_Assets_type': cur_n_assets}), ignore_index = True)

port_df['Real_N_assets'] = aux_df_real_n_assets['N_Assets_type'].tolist()

#Put the portfolio df with the number of shares in each period of time in the same format as the price series df
port_df_pivot = port_df.pivot(index = 'aux_date', columns = 'security', values = 'Real_N_assets').reset_index()
n_assets_type_table = pd.merge(align_price_series['aux_date'].to_frame(), port_df_pivot, how='left', on=['aux_date', 'aux_date']).fillna(method='ffill')

#Compute the portfolio value
port_val = align_price_series.copy()
for asset in etf_under_idx_list:
    port_val[asset] = port_val[asset] * n_assets_type_table[asset]
    
port_val['Total_Abs'] = port_val[etf_under_idx_list].sum(axis=1)

#Add to the index price series df an auxiliar date and generate the final df, where the index is the date and columns belong to the porfolio and HFRIFOF value
def_series = index_price_series.copy()

aligned_port_vals = []
for i_date in def_series['aux_date'].tolist():
    aligned_port_vals += [port_val.loc[port_val['aux_date'].isin([i_date])]['Total_Abs'].values[-1]]
    
#Calculate the cumulated total return of both value series
def_series['Port'] = aligned_port_vals
def_series = def_series.dropna(subset = ['Port'])
def_series[['Port', index_to_replicate]] = (def_series[['Port', index_to_replicate]].pct_change().fillna(0) + 1).cumprod() - 1

#Sort Columns
def_series = def_series[['DATE', 'Port', index_to_replicate, 'aux_date']]

#Plot the result
plot_def_series = def_series.drop(['aux_date'], axis=1).set_index('DATE')
result = bqv.ComparisonPlot(plot_def_series)

# Display the plot
result.show()